In [70]:
from rsa import ExtendedEuclideanAlgorithm, SquareAndMultiply, MillerRabin, RSA
import sympy
import random
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad


# RSA

## Extended Euclidean Algorithm

In [71]:
a = 39
m = 60
gcd, s, t = ExtendedEuclideanAlgorithm(a, m)
print(f'The GCD of {a} and {m} is {gcd}')
print(f's = {int(s)} and t = {int(t)} are the values so that sa + tm = gcd(a, m),'\
        f' indeed {int(s)}*{a} + {int(t)}*{m} = {gcd}\n')

a = 2560945
m = 98714510
gcd, s, t = ExtendedEuclideanAlgorithm(a, m)
print(f'The GCD of {a} and {m} is {gcd}')
print(f's = {int(s)} and t = {int(t)} are the values so that sa + tm = gcd(a, m),'\
        f' indeed ({int(s)}*{a}) + ({int(t)}*{m}) = {gcd}\n')

a = 0
m = 2560945
gcd, s, t = ExtendedEuclideanAlgorithm(a, m)
print(f'The GCD of {a} and {m} is {gcd}')
print(f's = {int(s)} and t = {int(t)} are the values so that sa + tm = gcd(a, m),'\
        f' indeed ({int(s)}*{a}) + ({int(t)}*{m}) = {gcd}\n')

a = 25
m = 25
gcd, s, t = ExtendedEuclideanAlgorithm(a, m)
print(f'The GCD of {a} and {m} is {gcd}')
print(f's = {int(s)} and t = {int(t)} are the values so that sa + tm = gcd(a, m),'\
        f' indeed ({int(s)}*{a}) + ({int(t)}*{m}) = {gcd}')


The GCD of 39 and 60 is 3
s = -3 and t = 2 are the values so that sa + tm = gcd(a, m), indeed -3*39 + 2*60 = 3

The GCD of 2560945 and 98714510 is 5
s = 4135267 and t = -107281 are the values so that sa + tm = gcd(a, m), indeed (4135267*2560945) + (-107281*98714510) = 5

The GCD of 0 and 2560945 is 2560945
s = 0 and t = 1 are the values so that sa + tm = gcd(a, m), indeed (0*0) + (1*2560945) = 2560945

The GCD of 25 and 25 is 25
s = 1 and t = 0 are the values so that sa + tm = gcd(a, m), indeed (1*25) + (0*25) = 25


## Square and Multiply

In [72]:
base = 4
exp = 5
mod = 35
y = SquareAndMultiply(base, exp, mod)
print(f'x^e \\ n = {y}')

x^e \ n = 9


## Miller Rabin Primality Test

In [73]:
n = 6701531651
isprime = MillerRabin(n, 1000)
if isprime:
    print(f'{n} is probably a prime number')
else:
    print(f'{n} is not a prime number')

n = 1962443363
isprime = MillerRabin(n, 1000)
if isprime:
    print(f'{n} is probably a prime number')
else:
    print(f'{n} is not a prime number') 

n = 786398548
isprime = MillerRabin(n, 1000)
if isprime:
    print(f'{n} is probably a prime number')
else:
    print(f'{n} is not a prime number')

n = 4
isprime = MillerRabin(n, 1000)
if isprime:
    print(f'{n} is probably a prime number')
else:
    print(f'{n} is not a prime number')

# n = product of 3 big prime numbers
n = 735495555426839*587000395258763*374906457293939
isprime = MillerRabin(n, 1000)
if isprime:
    print(f'{n} is probably a prime number')
else:
    print(f'{n} is not a prime number')



6701531651 is probably a prime number
1962443363 is probably a prime number
786398548 is not a prime number
4 is not a prime number
161860682384236698774917023853734785426608423 is not a prime number


## RSA implementation testing

In [74]:
##### Testing of the RSA class #####

# Inside the class the instructions in the DEBUG branches are executed
# Then p and q are equal to 335895919357171 and 744053548667773 respectively
bob = RSA(debug = True)
alice = RSA(n = bob.pub_key[0], e = bob.pub_key[1], debug = True)

# To simplify the debug process integers are used instead of bytes
plaintext = 3223675867980
print(f'\nThe plaintext is: {plaintext}')
ciphertext = alice.encrypt(plaintext)
print(f'The ciphertext is: {ciphertext}')
dec_plaintext = bob.decrypt(ciphertext)
print(f'The decrypted plaintext is: {dec_plaintext}')

# Full testing with random p and q and key length equal to 512 bits and
# bytes format for plaintext and ciphertext
bob = RSA(length = 512)
alice = RSA(n = bob.pub_key[0], e = bob.pub_key[1])
plaintext = random.randbytes(20)
print(f'\nThe plaintext is: {plaintext}')
ciphertext = alice.encrypt(plaintext)
print(f'The ciphertext is: {ciphertext}')
dec_plaintext = bob.decrypt(ciphertext)
print(f'The decrypted plaintext is: {dec_plaintext}')
 


The plaintext is: 3223675867980
The ciphertext is: 64091386761571093932794909611
The decrypted plaintext is: 3223675867980

The plaintext is: b'\x81\xdf\xf8p\x1a\xc0^\xe1\x0ep\x8b\x8d7\r\xad\xa4P\xdd\xa2\xc0'
The ciphertext is: b'BIk\xa8\xb9\xb6\xac\xb5\x1c\xb2e\x06\x82\xb5\x10\xac\xa9Rh\xe90 \x9e\xa6w\xe9\x82Tb\x0cdr$\xbd\xf2\xa9\xfc,R\xacf\xd0z\xe3\x82\xc1\xefT\x05\xa5S\xb9\xcac/uNXD\x17\xf1\x15#\xbc'
The decrypted plaintext is: b'\x81\xdf\xf8p\x1a\xc0^\xe1\x0ep\x8b\x8d7\r\xad\xa4P\xdd\xa2\xc0'


In [75]:
# The plaintext is chosen so that it longer than the key length. For this reason
# it cannot be encrypted. Let's see if an error is raised...
bob = RSA(length = 20)
print(bob.pub_key)
print(bob.priv_key)
alice = RSA(n = bob.pub_key[0], e = bob.pub_key[1])
plaintext = random.randbytes(30)
print(f'\nThe plaintext is: {plaintext}')
try:
    ciphertext = alice.encrypt(plaintext)
    print(f'The ciphertext is: {ciphertext}')
    dec_plaintext = bob.decrypt(ciphertext)
    print(f'The decrypted plaintext is: {dec_plaintext}')
except ValueError as e:
    print(e)

(568067, 5)
(568067, 453197)

The plaintext is: b'\x02\xe2\xf6 ^}\xa1~S\xc6\xdfJ\x8ao\xe2}Y\xa4KB\x15sSN\xfc\xdbL\x80Fa'
ERROR -> the plaintext to encrypt is too long. Try with a shorter one


## RSA and AES

In [76]:
with open('lorem_ipsum.txt', 'r') as f:
    aes_plaintext = f.read()
print(aes_plaintext[:70] + ' ...')

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmo ...


In [77]:
# AES.key_size is the list [16, 24, 32]. We select the third as
# key length (256 bits)

alice_aes_key = random.randbytes(AES.key_size[2])
iv = random.randbytes(16)
bob_rsa = RSA(length = 512)
alice_rsa = RSA(n = bob_rsa.pub_key[0], e = bob_rsa.pub_key[1])
bob_aes_key = bob_rsa.decrypt(alice_rsa.encrypt(alice_aes_key))
alice_aes = AES.new(alice_aes_key, AES.MODE_CBC, iv)
bob_aes = AES.new(bob_aes_key, AES.MODE_CBC, iv)
padded_aes_plaintext = pad(aes_plaintext.encode(), AES.block_size)
aes_ciphertext = alice_aes.encrypt(pad(aes_plaintext.encode(), AES.block_size))
dec_aes_plaintext = (unpad(bob_aes.decrypt(aes_ciphertext), AES.block_size)).decode()
if aes_plaintext == dec_aes_plaintext:
    print('Everything worked fine. Great job!')
else:
    print('Something went wrong. Check your implementation')


Everything worked fine. Great job!
